In [1]:
from common_tools import set_random_seeds
from data.tinyImageNet import tinyImageNetVague 

set_random_seeds(42)

data_dir = '/home/cxl173430/data/DATASETS/'
batch_size = 64

mydata = tinyImageNetVague(
            data_dir, 
            num_comp=1, 
            batch_size=batch_size,
            imagenet_hierarchy_path=data_dir,
            duplicate=True)

Loading TinyImageNet...
length of parent_to_subclass:  107
Total 29 Candidate superclasses: ['n04371563', 'n03419014', 'n04490091', 'n02924116', 'n02858304', 'n04379243', 'n07557434', 'n07611358', 'n07707451', 'n03748162', 'n03259505', 'n02898711', 'n03743902', 'n02796623', 'n09433442', 'n03278248', 'n04438304', 'n04147495', 'n04565375', 'n03880531', 'n02876657', 'n01767661', 'n01772222', 'n01942177', 'n02470325', 'n02370806', 'n02075296', 'n02121808', 'n01844917']
Vague classes: ['n03419014']
Vague classes nid: [['n02730930', 'n03763968']]
Vague classes ids: [[62, 119]]
Actual label sets
 R: [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [

In [2]:
from collections import Counter
labels_truth = []
labels_dup = []
for i in range(90490):
    labels_truth.append(mydata.train_loader.dataset[i][1])
    labels_dup.append(mydata.train_loader.dataset[i][2])

In [3]:
label_cnt_train_truth = Counter(labels_truth)
label_cnt_train_truth

Counter({0: 451,
         1: 451,
         2: 451,
         3: 451,
         4: 451,
         5: 451,
         6: 451,
         7: 451,
         8: 451,
         9: 451,
         10: 451,
         11: 451,
         12: 451,
         13: 451,
         14: 451,
         15: 451,
         16: 451,
         17: 451,
         18: 451,
         19: 451,
         20: 451,
         21: 451,
         22: 451,
         23: 451,
         24: 451,
         25: 451,
         26: 451,
         27: 451,
         28: 451,
         29: 451,
         30: 451,
         31: 451,
         32: 451,
         33: 451,
         34: 451,
         35: 451,
         36: 451,
         37: 451,
         38: 451,
         39: 451,
         40: 451,
         41: 451,
         42: 451,
         43: 451,
         44: 451,
         45: 451,
         46: 451,
         47: 451,
         48: 451,
         49: 451,
         50: 451,
         51: 451,
         52: 451,
         53: 451,
         54: 451,
         55: 451,
  

In [5]:
label_cnt_train_dup = Counter(labels_dup)
label_cnt_train_dup

Counter({0: 451,
         1: 451,
         2: 451,
         3: 451,
         4: 451,
         5: 451,
         6: 451,
         7: 451,
         8: 451,
         9: 451,
         10: 451,
         11: 451,
         12: 451,
         13: 451,
         14: 451,
         15: 451,
         16: 451,
         17: 451,
         18: 451,
         19: 451,
         20: 451,
         21: 451,
         22: 451,
         23: 451,
         24: 451,
         25: 451,
         26: 451,
         27: 451,
         28: 451,
         29: 451,
         30: 451,
         31: 451,
         32: 451,
         33: 451,
         34: 451,
         35: 451,
         36: 451,
         37: 451,
         38: 451,
         39: 451,
         40: 451,
         41: 451,
         42: 451,
         43: 451,
         44: 451,
         45: 451,
         46: 451,
         47: 451,
         48: 451,
         49: 451,
         50: 451,
         51: 451,
         52: 451,
         53: 451,
         54: 451,
         55: 451,
  

In [7]:
print(label_cnt_train_truth[62], label_cnt_train_truth[119])
print(label_cnt_train_dup[62], label_cnt_train_dup[119])

599 593
600 592


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
import torch
import copy
from backbones import HENN_EfficientNet
from backbones import EfficientNet_pretrain
import torch.nn.functional as F

# load the DNN model which has 0.86 singletonJS

In [3]:
result_path = "/home/cxl173430/data/uncertainty_Related/HENN_Git_VScode/HyperEvidentialNN_Results"
base_path = os.path.join(result_path, "tiny_baseline_DNN")
saved_path = os.path.join(base_path, "model_CrossEntropy.pt")
num_singles = 200
device = "cuda:9"
model = EfficientNet_pretrain(num_singles)
checkpoint = torch.load(saved_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])

model_best_from_valid = copy.deepcopy(model)
model_best_from_valid.load_state_dict(checkpoint["model_state_dict_best"]) 

Loaded pretrained weights for efficientnet-b3


<All keys matched successfully>

In [4]:
def calculate_metrics(output, labels, R, cutoff, detNN=True):
    if detNN:
        p_exp = F.softmax(output, dim=1)
    else:
        alpha = torch.add(output, 1)
        alpha_sum = torch.sum(alpha, dim=1)
        p_exp = torch.div(alpha, alpha_sum[:, None])
    
    # Get the predicted labels
    predicted_labels = torch.argmax(p_exp, dim=1)
    num_singles = output.shape[1]
    correct_vague = 0.0
    correct_nonvague = 0.0
    vague_total = 0
    nonvague_total = 0
    predSet_or_not = []
    selected_idx = []
    for i in range(len(labels)):
        indices = (p_exp[i] >= cutoff).nonzero(as_tuple=True)[0]
        predicted_set = set(indices.tolist())

        if len(predicted_set) == 1:
            predicted_set = set(R[predicted_labels[i].item()])
            predSet_or_not.append(0) # singleton
            selected_idx.append(True)
        else:
            predSet_or_not.append(1)
            selected_idx.append(False)

        ground_truth_set = set(R[labels[i].item()])
        intersect = predicted_set.intersection(ground_truth_set)
        union = predicted_set.union(ground_truth_set)
        if len(predicted_set) == 1:
            correct_nonvague += float(len(intersect)) / len(union)
            nonvague_total += 1
        else:
            correct_vague += float(len(intersect)) / len(union)
            vague_total += 1
    stat_result = [correct_nonvague, correct_vague, nonvague_total, vague_total]
    
    predSet_or_not = torch.tensor(predSet_or_not) #1:vague, 0：non-vague
    # check precision, recall, f-score for composite classes
#     prec_r_f = precision_recall_f_v1(labels, predSet_or_not, num_singles)
    return stat_result, selected_idx

In [5]:
from torch.utils.data import sampler, random_split, DataLoader
test_set = mydata.test_loader.dataset
test_loader = DataLoader(test_set, batch_size=1, num_workers=1, pin_memory=True)
detNN = True 


# model = model.to(device)
model_best_from_valid = model_best_from_valid.to(device)
model = model_best_from_valid

R = mydata.R 
num_singles = 200
num_comp = 1 

#     cutoff = get_cutoff(model, val_loader, R, device, detNN=detNN)
cutoff = 0.25
print(f"### selected cutoff: {cutoff}")
model.eval()
outputs_all = []
labels_all = []
preds_all = []
with torch.no_grad():
    for batch in test_loader:
        images, _, labels = batch
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        preds = output.argmax(dim=1)

        outputs_all.append(output)
        labels_all.append(labels)
        preds_all.append(preds)

outputs_all = torch.cat(outputs_all, dim=0)
labels_all = torch.cat(labels_all, dim=0)
stat_result, selected_idx = calculate_metrics(outputs_all, labels_all, R, cutoff, detNN=detNN)

avg_js_nonvague = stat_result[0] / (stat_result[2]+1e-10)
avg_js_vague = stat_result[1] / (stat_result[3]+1e-10)
overall_js = (stat_result[0] + stat_result[1])/(stat_result[2] + stat_result[3]+1e-10)
js_result = [overall_js, avg_js_vague, avg_js_nonvague]


### selected cutoff: 0.25


In [6]:
js_result

[0.8185499999999918, 0.3465189873417356, 0.8679849756959692]

# select the examples corresponds to 0.86 singletonJS

In [ ]:
selected_idx

# load the HENN model which has 0.81 singletonJS

In [7]:
result_path = "/home/cxl173430/data/uncertainty_Related/HENN_Git_VScode/HyperEvidentialNN_Results"
base_path = os.path.join(result_path, "exp4/lambda_1")
saved_path = os.path.join(base_path, "model_uncertainty_digamma.pt")

device = "cuda:8"
num_both = 201
model = HENN_EfficientNet(num_both)
checkpoint = torch.load(saved_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])

model_best_from_valid = copy.deepcopy(model)
model_best_from_valid.load_state_dict(checkpoint["model_state_dict_best"]) 

Loaded pretrained weights for efficientnet-b3


<All keys matched successfully>

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics_ENN(output, labels, selected_idx, R):
    GTs = []
    Predicteds = []

    correct_vague = 0.0
    correct_nonvague = 0.0
    vague_total = 0
    nonvague_total = 0
    corr_selected = 0.
    for i in range(len(labels)):
        k = labels[i].item()
        predicted_set = set(R[torch.argmax(output[i])])
        Predicteds.append(predicted_set)

        ground_truth_set = set(R[k])
        GTs.append(ground_truth_set)
        
        intersect = predicted_set.intersection(ground_truth_set)
        union = predicted_set.union(ground_truth_set)
        if len(predicted_set) == 1:
            correct_nonvague += float(len(intersect)) / len(union)
            nonvague_total += 1
        else:
            correct_vague += float(len(intersect)) / len(union)
            vague_total += 1
        if selected_idx[i] == True:
            corr_selected += float(len(intersect)) / len(union)
    
    stat_result = [correct_nonvague, correct_vague, nonvague_total, vague_total] #todo check this with calculate_metric
    GT_Pred_res = [GTs, Predicteds]
    
    selected_num = torch.sum(torch.tensor(selected_idx))
    print("#selected: ", selected_num)
    print("### singlJS': ", corr_selected/selected_num)
    return stat_result, GT_Pred_res

In [10]:
def acc_subset(idx, labels_true, labels_pred):
    labels_true_subs = labels_true[idx]
    labels_pred_subs = labels_pred[idx]
    corr_subs = torch.sum(labels_true_subs == labels_pred_subs).item()
    acc_subs = corr_subs / len(labels_true_subs)
    return acc_subs

model_best_from_valid = model_best_from_valid.to(device)
model = model_best_from_valid

model.eval()
outputs_all = []
labels_all = []
preds_all = []

correct = 0
with torch.no_grad():
    for batch in test_loader:
        images, _, labels = batch
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        preds = output.argmax(dim=1)
        correct += torch.sum(preds == labels.data)
        outputs_all.append(output)
        labels_all.append(labels)
        preds_all.append(preds)
        

outputs_all = torch.cat(outputs_all, dim=0)
labels_all = torch.cat(labels_all, dim=0)
preds_all = torch.cat(preds_all, dim=0)
acc = correct / len(labels_all)

# calculate the accuracy among singleton examples
# acc of composite examples
comp_idx = labels_all > num_singles-1
acc_comp = acc_subset(comp_idx, labels_all, preds_all)

# acc of singleton examples
singl_idx = labels_all < num_singles
acc_singl = acc_subset(singl_idx, labels_all, preds_all)

stat_result, GT_Pred_res = calculate_metrics_ENN(outputs_all, labels_all, selected_idx, R)

avg_js_nonvague = stat_result[0] / (stat_result[2]+1e-10)
avg_js_vague = stat_result[1] / (stat_result[3]+1e-10)
overall_js = (stat_result[0] + stat_result[1])/(stat_result[2] + stat_result[3]+1e-10)

js_result = [overall_js, avg_js_vague, avg_js_nonvague]
js_result

#selected:  tensor(9052)
### singlJS':  tensor(0.8509)


[0.8169999999999918, 0.9999999999968749, 0.8164125200641973]

# cifar100

In [1]:
from data.cifar100 import CIFAR100Vague 

data_dir = '/home/cxl173430/DATASETS/'
batch_size = 64

mydata = CIFAR100Vague(
            data_dir, 
            num_comp=1, 
            batch_size=batch_size,
#             imagenet_hierarchy_path=data_dir,
            duplicate=True)

Loading CIFAR100...
Files already downloaded and verified
/home/cxl173430/DATASETS/cifar100_henn/
Hierarchical labels:

aquatic_mammals :  ['beaver', 'dolphin', 'otter', 'seal', 'whale']
fish :  ['aquarium_fish', 'flatfish', 'ray', 'shark', 'trout']
flowers :  ['orchid', 'poppy', 'rose', 'sunflower', 'tulip']
food_containers :  ['bottle', 'bowl', 'can', 'cup', 'plate']
fruit_and_vegetables :  ['apple', 'mushroom', 'orange', 'pear', 'sweet_pepper']
household_electrical_devices :  ['clock', 'keyboard', 'lamp', 'telephone', 'television']
household_furniture :  ['bed', 'chair', 'couch', 'table', 'wardrobe']
insects :  ['bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach']
large_carnivores :  ['bear', 'leopard', 'lion', 'tiger', 'wolf']
large_man-made_outdoor_things :  ['bridge', 'castle', 'house', 'road', 'skyscraper']
large_natural_outdoor_scenes :  ['cloud', 'forest', 'mountain', 'plain', 'sea']
large_omnivores_and_herbivores :  ['camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo

In [2]:
import torch
a = torch.tensor([1,2,3,4,5])
idx = a>3
print(idx)

tensor([False, False, False,  True,  True])


In [3]:
a[idx]

tensor([4, 5])

In [4]:
b = torch.tensor([6,7,8,9,0])
b[idx]

tensor([9, 0])

In [5]:
import random
random.seed(2)
random.sample([1,2,3], 2)

[1, 3]

In [6]:
random.sample([1,2,3], 2)

[1, 2]

In [7]:
random.sample([1,2,3], 2)

[1, 2]

In [8]:
random.sample([1,2,3], 2)

[2, 1]

In [9]:
random.seed(2)
random.sample([1,2,3], 2)

[1, 3]

In [10]:
random.seed(2)
random.sample([1,2,3], 2)

[1, 3]